# logging

> Data Models

In [ ]:
#| default_exp log_utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import sys
import logging
import sys, io
from contextlib import redirect_stdout, redirect_stderr

In [ ]:
#| export

def in_ipython() -> bool:
    try:
        from IPython import get_ipython  # noqa
        return get_ipython() is not None
    except Exception:
        return False

def setup_logger(name: str = "app", level: int = logging.INFO) -> logging.Logger:
    """
    Create a logger that works in Jupyter (threads) and normal Python scripts.
    - In notebooks: writes to sys.__stdout__ to avoid IPython's capture issues
    - In scripts:   writes to sys.stdout
    """
    stream = sys.__stdout__ if in_ipython() else sys.stdout

    logger = logging.getLogger(name)
    logger.setLevel(level)

    # Clear existing handlers to avoid duplicate logs when re-running a cell
    logger.handlers.clear()
    logger.propagate = False

    handler = logging.StreamHandler(stream=stream)
    handler.setLevel(level)
    handler.flush = handler.flush  # explicit, but ensures availability

    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(threadName)s | %(message)s")
    handler.setFormatter(fmt)

    logger.addHandler(handler)
    return logger


In [ ]:
#| export

def call_with_visible_output(fn, *args, **kwargs):
    """
    Run fn(*args, **kwargs) while capturing stdout/stderr, then
    mirror that output to the real stdout/stderr so it appears in
    Jupyter and in normal scripts.
    """
    buf = io.StringIO()
    err = io.StringIO()
    with redirect_stdout(buf), redirect_stderr(err):
        result = fn(*args, **kwargs)
    out_text = buf.getvalue()
    err_text = err.getvalue()
    if out_text:
        sys.__stdout__.write(out_text)
        sys.__stdout__.flush()
    if err_text:
        sys.__stderr__.write(err_text)
        sys.__stderr__.flush()
    return result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()